<a href="https://colab.research.google.com/github/ravbuilds/nvidia-nat/blob/main/climate_science_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Climate Science Chatbot**
This notebook creates and runs the NAT workflow to define agents capable of fetching, analyzing and visualizing real NOAA climate data.

###**Install Libraries**
Steps:
1. Install Nemo Agent Toolkit and LangChain dependency
2. Load API Keys - using .env file
3. Create NAT configuration
4. Run the Workflow

In [1]:
%%capture
! pip install nvidia-nat
! pip install "nvidia-nat[langchain]"

In [11]:
# Load API Keys.
# In this notebook I am loading the API keys using Google Colab Secrets
# Other options include loading a temporary .env local file to Colab runtime.

from google.colab import userdata
import os

NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')
MODEL_BASE_URL = "https://integrate.api.nvidia.com/v1"

###**Create NAT Configuration**

In [12]:
%%writefile config.yaml
llms:
  climate_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    base_url: $MODEL_BASE_URL
    api_key: $NVIDIA_API_KEY
    temperature: 0.7
    max_tokens: 2048

workflow:
  _type: chat_completion
  llm_name: climate_llm
  system_prompt: |
    You are a knowledgeable climate science assistant. You help users understand
    climate data, weather patterns, and global temperature trends. Be accurate,
    informative, and cite scientific consensus when appropriate.

Overwriting config.yaml


###**Test Workflow**
We will use the `NAT CLI` to run our workflow and test it.

In [13]:
!nat run \
  --config_file config.yaml \
  --input "What is the difference between weather and climate?"

2026-02-22 17:24:09 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yaml'
2026-02-22 17:24:09 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-02-22 17:24:09 - INFO     - nat.runtime.session:298 - Shared workflow built (entry_function=None)
/usr/local/lib/python3.12/dist-packages/nat/builder/function.py:375: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .te

In [14]:
!nat run \
  --config_file config.yaml \
  --input "List temperature anamolies for the top 5 warmest countries in 2025?"

2026-02-22 17:26:18 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yaml'
2026-02-22 17:26:18 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-02-22 17:26:18 - INFO     - nat.runtime.session:298 - Shared workflow built (entry_function=None)
/usr/local/lib/python3.12/dist-packages/nat/builder/function.py:375: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .te

### Deploying as an API
In production we will run `nat serve` in a terminal. Since we are running in JupyterNB, we have to run it as a subprocess.<br/>
> *nat serve will serve this agentic workflow as a self-contained api*

In [15]:
import subprocess
import time

nat_serve = subprocess.Popen(
    ["nat", "serve", "--config_file", "config.yaml"],
    stderr=subprocess.DEVNULL
)

# Run the let it print to console
time.sleep(25)
print("\n Server up and running in the background")


 Server up and running in the background


### Test the API

In [20]:
import requests
import json

response = requests.post(
    "http://localhost:8000/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    json={
        "messages": [
            {
                "role": "user",
                "content": "What causes El Nino and how does it affect global weather"
            }
        ],
        "stream": False
    }
)

if response and response.status_code == 200:
  result = response.json()
  print(result['choices'][0]['message']['content'])
else:
  print(f"Error: {response.status_code}")
  print(response.text)

El Niño is a complex weather phenomenon that occurs when the surface temperature of the Pacific Ocean warms up more than usual, typically by 0.5-1°C (0.9-1.8°F), near the equator off the coast of South America. This warming of the ocean water is caused by changes in the trade winds and the thermocline (the layer of water where the temperature changes rapidly with depth).

 Normally, the trade winds blow from east to west along the equator, pushing warm water towards Asia and allowing cooler water to rise up from the depths of the ocean near South America. However, during an El Niño event, the trade winds weaken or even reverse direction, allowing the warm water to spread back towards the eastern Pacific. This warming of the ocean water can have significant effects on global weather patterns.

El Niño affects global weather in several ways:

1. **Heavy rainfall in South America**: The warming of the ocean water leads to increased evaporation, which in turn fuels heavy rainfall in countr

### Launching a UI
Launch a UI to interact with the agent

In [21]:
# Example of using ipywidgets in Colab
from ipywidgets import interact, widgets


interactive(children=(Dropdown(description='column', options=('A', 'B', 'C'), value='A'), Output()), _dom_clas…